## Pandas - Applications in Finance

In [1]:
import numpy as np
import pandas as pd
import datetime, calendar

import matplotlib.pyplot as plt
from mpl_finance import candlestick_ohlc

import matplotlib.dates as mdates

np.set_printoptions(suppress=True)

ModuleNotFoundError: No module named 'mpl_finance'

In [ ]:
datafile = 'http://people.bu.edu/kalathur/datasets/AAPL.csv'

df = pd.read_csv(datafile, index_col='Date', parse_dates=True)
df = df.round(2)
df

In [ ]:
df.mean().round(2)

**Volume Weighted Average Price (VWAP)**

In [ ]:
vwap = np.average(df['Adj Close'], weights=df['Volume'])
print("VWAP =", vwap)

### Averages by Day of Week

In [ ]:
for freq in ['W-MON', 'W-TUE', 'W-WED', 'W-THU', 'W-FRI']:
    print(freq, "Average", df.resample(freq).asfreq().dropna()['Adj Close'].mean().round(2))

In [ ]:
result = df.groupby(df.index.dayofweek)['Adj Close'].mean().round(2)
result

In [ ]:
result.index = [calendar.day_name[index] for index in result.index]
result

**Interpreting Moving Averages**

https://www.tradingview.com/wiki/Moving_Average

## Simple Moving Average (SMA)
 - For analyzing time-series data
 - Moving window of N periods
 - Mean of values inside the window
 - an unweighted moving average

In [ ]:
# 20-day moving window

N = 20

In [ ]:
newDF = df.copy()

In [ ]:
newDF['SMA'] = df['Adj Close'].rolling(window=N).mean().round(2)
newDF

In [ ]:
newDF['mDate'] = mdates.date2num(df.index)
newDF
    

In [ ]:
olhc = newDF[['mDate', 'Open', 'High', 'Low', 'Adj Close']].copy().values
olhc[:5]

In [ ]:
fig, ax = plt.subplots(1, figsize=(12,8))

candlestick_ohlc(ax, olhc)

plt.plot(newDF['mDate'], newDF['Adj Close'], label='Data')
plt.plot(newDF['mDate'], newDF['SMA'], '--', lw=2.0, label='SMA')

ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))

plt.title("20 Day Moving Average")
plt.xlabel("Days")
plt.ylabel("Price ($)")
plt.grid()
plt.legend()
plt.show()



In [ ]:
# Show plot for lastN values only

lastN = 50

fig, ax = plt.subplots(1, figsize=(12,8))

candlestick_ohlc(ax, olhc[-lastN:])

plt.plot(newDF['mDate'][-lastN:], newDF['Adj Close'][-lastN:], label='Data')
plt.plot(newDF['mDate'][-lastN:], newDF['SMA'][-lastN:], '--', lw=2.0, label='SMA')

ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))

plt.title("20 Day Moving Average")
plt.xlabel("Days")
plt.ylabel("Price ($)")
plt.grid()
plt.legend()
plt.show()

## Weighted Moving Average (WMA)

In [ ]:
# 20-day moving window

N = 20

In [ ]:
weights = np.arange(1, N+1)
print(weights)

In [ ]:
# Return closure function to be applied on each group

def f(w):
    def g(x):
        return np.sum(x*w)/sum(w)
    return g

In [ ]:
newDF['WMA'] = df[['Adj Close']].rolling(window=N).apply(f(weights))
newDF

In [ ]:
fig, ax = plt.subplots(1, figsize=(12,8))

plt.plot(newDF.index, newDF['Adj Close'], label='Data')
plt.plot(newDF.index, newDF['SMA'], '--', lw=2.0, label='SMA')
plt.plot(newDF.index, newDF['WMA'], '-.', lw=2.0, label='WMA')

plt.title("20 Day Moving Average")
plt.xlabel("Days")
plt.ylabel("Price ($)")
plt.grid()
plt.legend()
plt.show()


## Exponential Moving Average (EMA)
 - For analyzing time-series data
 - Alternative to SMA
 - Moving window of N periods
 - Uses exponentially decreasing weights
 - Gives higher weights to recent prices
 - weighted averages are calculated using weights $(1-alpha)**(n-1), (1-alpha)**(n-2), …, 1-alpha, 1$
 

In [ ]:
newDF['EMA'] = df[['Adj Close']].ewm(alpha=0.5, min_periods=N).mean()
newDF

In [ ]:
fig, ax = plt.subplots(1, figsize=(12,8))

plt.plot(newDF.index, newDF['Adj Close'], label='Data')
plt.plot(newDF.index, newDF['SMA'], '--', lw=2.0, label='SMA')
plt.plot(newDF.index, newDF['WMA'], '-.', lw=2.0, label='WMA')
plt.plot(newDF.index, newDF['EMA'], ':',  lw=2.0, label="EMA")


plt.title("20 Day Moving Average")
plt.xlabel("Days")
plt.ylabel("Price ($)")
plt.grid()
plt.legend()
plt.show()